# Orientation Tuning Analysis Pipeline

This notebook demonstrates the complete workflow for analyzing 2-photon calcium imaging data:
1. FOV configuration setup
2. Suite2P trace extraction
3. Trial organization and response calculation
4. Orientation/direction tuning analysis
5. Comprehensive visualization

## Requirements
```bash
pip install numpy scipy matplotlib h5py
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from fov_config_suite2p import FOV
from fov_utils import create_fov_from_stimfile, export_fov_to_dict
from ophys_analysis import (
    extract_suite2p_traces,
    save_extraction_hdf5,
    load_extraction_hdf5,
    plot_population_summary,
    plot_orientation_map,
    plot_tuning_distributions,
    plot_cell_tuning_curve,
    create_full_analysis_report,
    get_tuning_madineh,
)

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

## 1. FOV Configuration

Create FOV configuration from stimulus file and set required parameters.

In [ ]:
# Example paths - UPDATE THESE FOR YOUR DATA
data_dir = Path(r"X:\Experimental_Data\BrainImaging\20251113_Derrick")

# IMPORTANT: Spk2File should be the ACTUAL directory number, not a list index!
# For directory t00016/, use Spk2File=[16], not [0]
# Check your data directory to find the correct t* folder number

# Create FOV from stimulus file
fov = create_fov_from_stimfile(
    stimfile=None,      # Auto-detect .py file in Spk2 subdirectory
    TifStack_path=str(data_dir),
    ImagingFile=[0],    # Imaging file indices
    Spk2File=[16],      # Actual Spk2 directory number (for t00016/)
    factor=1,           # Optional parameters
    brain_region='V1'
)

# Display configuration
print("\nFOV Configuration:")
print("="*50)
for key, val in export_fov_to_dict(fov).items():
    print(f"{key:20s}: {val}")

## 2. Extract Suite2P Traces

Load Suite2P output and organize into trial structure.

In [ ]:
# Run extraction
ce = extract_suite2p_traces(fov, fnum=0)

# Print summary
ce.print_summary()

# Get basic statistics
n_cells = len(ce.cells)
n_responsive = sum(c.ROI_responsiveness for c in ce.cells)

print(f"\n{'='*50}")
print(f"Total cells extracted: {n_cells}")
print(f"Visually responsive: {n_responsive} ({100*n_responsive/n_cells:.1f}%)")
print(f"Non-responsive: {n_cells - n_responsive}")

## 3. Save Extraction Results

Save to HDF5 for later analysis.

In [ ]:
# Save to HDF5
output_file = data_dir / "extraction_results.h5"
save_extraction_hdf5(ce, str(output_file))

# Verify by loading
ce_loaded = load_extraction_hdf5(str(output_file))
print(f"\nVerified: Loaded {len(ce_loaded.cells)} cells from saved file")

## 4. Population Summary

Visualize overall population statistics.

In [ ]:
plot_population_summary(ce)

## 5. Tuning Distributions

Analyze orientation and direction selectivity across the population.

In [ ]:
plot_tuning_distributions(ce)

## 6. Orientation/Direction Preference Maps

Visualize spatial organization of tuning preferences.

In [ ]:
# Optional: Load FOV image for background
# fov_image = np.load(data_dir / "mean_image.npy")
fov_image = None

plot_orientation_map(ce, fov_image=fov_image)

## 7. Individual Cell Analysis

Examine tuning curves for individual responsive cells.

In [ ]:
# Get responsive cell indices
responsive_indices = [i for i, c in enumerate(ce.cells) if c.ROI_responsiveness]
print(f"Analyzing {len(responsive_indices)} responsive cells\n")

# Plot first 5 responsive cells
for idx in responsive_indices[:5]:
    cell = ce.cells[idx]
    
    # Get stimulus info
    n_dirs = len(cell.uniqStims) - 1  # Exclude blank
    stimInfo = np.arange(0, 360, 360/n_dirs)
    
    # Get tuning metrics
    tuning, _, fitdata = get_tuning_madineh(cell.condition_response[:n_dirs], stimInfo)
    
    # Plot
    plot_cell_tuning_curve(cell, stimInfo, tuning, fitdata, idx, fov.stim_dur)
    
    # Print metrics
    print(f"Cell {idx}:")
    print(f"  Preferred Orientation: {tuning['pref_ort_fit']:.1f}°")
    print(f"  OTI: {tuning['oti_vec']:.3f}")
    print(f"  Preferred Direction: {tuning['pref_dir_fit']:.1f}°")
    print(f"  DTI: {tuning['dti_vec']:.3f}")
    print(f"  Bandwidth: {tuning['fit_bandwidth']:.1f}°")
    print(f"  Fit correlation: {tuning['fit_r']:.3f}")
    print()

## 8. Generate Complete Analysis Report

Create all analysis figures and save to output directory.

In [ ]:
# Create output directory
output_dir = data_dir / "analysis_report"

# Generate all plots
create_full_analysis_report(ce, fov_image=fov_image, output_dir=str(output_dir))

print(f"\nAnalysis report saved to: {output_dir}")

## 9. Advanced Analysis Examples

### Filter cells by tuning properties

In [ ]:
# Find orientation-selective cells (high OTI, low DTI)
orientation_selective = []

for i, cell in enumerate(ce.cells):
    if cell.ROI_responsiveness:
        n_dirs = len(cell.uniqStims) - 1
        stimInfo = np.arange(0, 360, 360/n_dirs)
        try:
            tuning, _, _ = get_tuning_madineh(cell.condition_response[:n_dirs], stimInfo)
            
            # Criteria: OTI > 0.3, DTI < 0.3
            if tuning['oti_fit'] > 0.3 and tuning['dti_fit'] < 0.3:
                orientation_selective.append(i)
        except:
            pass

print(f"Found {len(orientation_selective)} orientation-selective cells")
print(f"Indices: {orientation_selective[:10]}...")  # Show first 10

In [ ]:
# Find direction-selective cells (high DTI)
direction_selective = []

for i, cell in enumerate(ce.cells):
    if cell.ROI_responsiveness:
        n_dirs = len(cell.uniqStims) - 1
        stimInfo = np.arange(0, 360, 360/n_dirs)
        try:
            tuning, _, _ = get_tuning_madineh(cell.condition_response[:n_dirs], stimInfo)
            
            # Criteria: DTI > 0.5
            if tuning['dti_fit'] > 0.5:
                direction_selective.append(i)
        except:
            pass

print(f"Found {len(direction_selective)} direction-selective cells")
print(f"Indices: {direction_selective[:10]}...")  # Show first 10

### Analyze spatial clustering of orientation preferences

In [ ]:
from scipy.spatial.distance import pdist, squareform

# Get tuning and position data for responsive cells
positions = []
orientations = []

for i, cell in enumerate(ce.cells):
    if cell.ROI_responsiveness and cell.xPos is not None:
        n_dirs = len(cell.uniqStims) - 1
        stimInfo = np.arange(0, 360, 360/n_dirs)
        try:
            tuning, _, _ = get_tuning_madineh(cell.condition_response[:n_dirs], stimInfo)
            positions.append([cell.xPos, cell.yPos])
            orientations.append(tuning['pref_ort_fit'])
        except:
            pass

positions = np.array(positions)
orientations = np.array(orientations)

# Calculate spatial distances
spatial_dist = squareform(pdist(positions))

# Calculate orientation differences (accounting for 180° periodicity)
ori_diff = np.abs(orientations[:, None] - orientations[None, :])
ori_diff = np.minimum(ori_diff, 180 - ori_diff)

# Plot relationship
fig, ax = plt.subplots(figsize=(8, 6))

# Flatten arrays and remove diagonal
mask = ~np.eye(len(positions), dtype=bool)
spatial_flat = spatial_dist[mask]
ori_flat = ori_diff[mask]

ax.scatter(spatial_flat, ori_flat, alpha=0.1, s=5)
ax.set_xlabel('Spatial Distance (pixels)')
ax.set_ylabel('Orientation Difference (°)')
ax.set_title('Spatial vs Orientation Similarity')
ax.set_ylim([0, 90])

# Calculate correlation
from scipy.stats import pearsonr
r, p = pearsonr(spatial_flat, ori_flat)
ax.text(0.02, 0.98, f'r = {r:.3f}, p = {p:.2e}',
        transform=ax.transAxes, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

if r > 0:
    print("\nPositive correlation suggests orientation clustering")
elif r < 0:
    print("\nNegative correlation suggests orientation salt-and-pepper organization")
else:
    print("\nNo clear spatial organization of orientation preferences")

## 10. Export Results for Further Analysis

In [ ]:
# Create summary CSV with tuning metrics
import pandas as pd

results = []
for i, cell in enumerate(ce.cells):
    if cell.ROI_responsiveness:
        n_dirs = len(cell.uniqStims) - 1
        stimInfo = np.arange(0, 360, 360/n_dirs)
        try:
            tuning, _, _ = get_tuning_madineh(cell.condition_response[:n_dirs], stimInfo)
            results.append({
                'cell_id': i,
                'x_pos': cell.xPos,
                'y_pos': cell.yPos,
                'pref_orientation': tuning['pref_ort_fit'],
                'pref_direction': tuning['pref_dir_fit'],
                'oti_vec': tuning['oti_vec'],
                'oti_fit': tuning['oti_fit'],
                'dti_vec': tuning['dti_vec'],
                'dti_fit': tuning['dti_fit'],
                'bandwidth': tuning['fit_bandwidth'],
                'fit_r': tuning['fit_r'],
            })
        except:
            pass

df = pd.DataFrame(results)
df.to_csv(data_dir / "tuning_metrics.csv", index=False)
print(f"Saved tuning metrics for {len(df)} cells to tuning_metrics.csv")
df.head()